In [4]:
!git clone https://github.com/OpenGVLab/InternImage

Cloning into 'InternImage'...
remote: Enumerating objects: 1216, done.
remote: Counting objects: 100% (351/351), done.
remote: Compressing objects: 100% (218/218), done.
remote: Total 1216 (delta 198), reused 208 (delta 131), pack-reused 865
Receiving objects: 100% (1216/1216), 22.84 MiB | 17.99 MiB/s, done.
Resolving deltas: 100% (508/508), done.


In [8]:
!pip install -U openmim
!mim install mmcv-full==1.7.1
!pip install timm==0.6.11 mmdet==2.28.1
!pip install opencv-python termcolor yacs pyyaml scipy

Looking in links: https://download.openmmlab.com/mmcv/dist/cu117/torch2.0.0/index.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 548.7/548.7 kB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 29.5 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: timm
    Found existing installation: timm 0.9.2
    Uninstalling timm-0.9.2:
      Successfully uninstalled timm-0.9.2


In [13]:
%cd detection
%cd ./ops_dcnv3
!sh ./make.sh
# unit test (should see all checking is True)
!python test.py

[Errno 2] No such file or directory: 'detection'
/home/alc/TIL-2023/CV/InternImage/InternImage/detection
/home/alc/TIL-2023/CV/InternImage/InternImage/detection/ops_dcnv3
running build
running build_py
creating build
creating build/lib.linux-x86_64-3.10
creating build/lib.linux-x86_64-3.10/functions
copying functions/dcnv3_func.py -> build/lib.linux-x86_64-3.10/functions
copying functions/__init__.py -> build/lib.linux-x86_64-3.10/functions
creating build/lib.linux-x86_64-3.10/modules
copying modules/dcnv3.py -> build/lib.linux-x86_64-3.10/modules
copying modules/__init__.py -> build/lib.linux-x86_64-3.10/modules
running build_ext
/home/alc/venvs/til2023/lib/python3.10/site-packages/torch/utils/cpp_extension.py:388: UserWarning: The detected CUDA version (11.8) has a minor version mismatch with the version that was used to compile PyTorch (11.7). Most likely this shouldn't be a problem.
  warnings.warn(CUDA_MISMATCH_WARN.format(cuda_str_version, torch.version.cuda))
/home/alc/venvs/til

In [75]:
!sh dist_train.sh configs/coco/cascade_internimage_l_fpn_3x_coco_custom.py 1

/home/alc/TIL-2023/CV/InternImage/InternImage/detection
/home/alc/venvs/til2023/lib/python3.10/site-packages/torch/distributed/launch.py:181: FutureWarning: The module torch.distributed.launch is deprecated
and will be removed in future. Use torchrun.
Note that --use-env is set by default in torchrun.
If your script expects `--local-rank` argument to be set, please
change it to read from `os.environ['LOCAL_RANK']` instead. See 
https://pytorch.org/docs/stable/distributed.html#launch-utility for 
further instructions

  warnings.warn(
/home/alc/venvs/til2023/lib/python3.10/site-packages/mmcv/__init__.py:20: UserWarning: On January 1, 2023, MMCV will release v2.0.0, in which it will remove components related to the training process and add a data transformation module. In addition, it will rename the package names mmcv to mmcv-lite and mmcv-full to mmcv. See https://github.com/open-mmlab/mmcv/blob/master/docs/en/compatibility.md for more details.
  warnings.warn(
/home/alc/venvs/til2023/

In [72]:
# generate fake json label for test to please mmdet
with open('/home/alc/TIL-2023/CV/RT-DETR/dataset/val/val_with_seg.json') as f:
    data = json.load(f)
sample_anno = data['annotations'][-1].copy()
print(sample_anno)

for i in range(800, 1600):
    data['images'].append({'file_name': f'image_{str(i).zfill(4)}.png', 'height': 720, 'width': 1280, 'id': i})
    sample_anno['id'] = sample_anno['id'] + 1
    sample_anno['image_id'] = i
    data['annotations'].append(sample_anno.copy())
print(data['annotations'][-1])
with open('/home/alc/TIL-2023/CV/RT-DETR/dataset/test/test_with_seg.json', 'w+') as f:
    json.dump(data, f)

{'id': 1699, 'image_id': 799, 'bbox': [86.0, 364.0, 179.0, 164.0], 'area': 29356, 'iscrowd': 0, 'category_id': 1, 'segmentation': [[86.0, 364.0, 265.0, 364.0, 265.0, 528.0, 86.0, 528.0]]}
{'id': 2499, 'image_id': 1599, 'bbox': [86.0, 364.0, 179.0, 164.0], 'area': 29356, 'iscrowd': 0, 'category_id': 1, 'segmentation': [[86.0, 364.0, 265.0, 364.0, 265.0, 528.0, 86.0, 528.0]]}


In [79]:
!python test.py \
    configs/coco/cascade_internimage_l_fpn_3x_coco_custom.py \
    work_dirs/cascade_internimage_l_fpn_3x_coco_custom/InternImage-L epoch_12.pth \
    --show-dir epoch12_plots --show-score-thr 0.9 --out epoch_12.pkl

/home/alc/venvs/til2023/lib/python3.10/site-packages/mmcv/__init__.py:20: UserWarning: On January 1, 2023, MMCV will release v2.0.0, in which it will remove components related to the training process and add a data transformation module. In addition, it will rename the package names mmcv to mmcv-lite and mmcv-full to mmcv. See https://github.com/open-mmlab/mmcv/blob/master/docs/en/compatibility.md for more details.
  warnings.warn(
!!!!!!!!!!!!!!!!!!1
!!!!!!!!!!!!!!!!!!2
!!!!!!!!!!!!!!!!!!3
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
2023-06-02 11:00:04,760 - mmdet - INFO - using core type: DCNv3
2023-06-02 11:00:04,760 - mmdet - INFO - using activation layer: GELU
2023-06-02 11:00:04,760 - mmdet - INFO - using main norm layer: LN
2023-06-02 11:00:04,760 - mmdet - INFO - using dpr: linear, 0.4
2023-06-02 11:00:04,760 - mmdet - INFO - level2_post_norm: False
2023-06-02 11:00:04,760 - mmdet - INFO - level2_post_norm_block_ids: None
2023-06-02 11:00: